In [40]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import sample_rasters as sr

In [28]:
aois = ['campus_lagoon','carpinteria','gaviota','point_conception']
years = [2012, 2014, 2016, 2018, 2020]

In [62]:
aoi_col = []
year_col = []
n_iceplant = []
n_polygons = []
area_ice = []
total_area = []

for aoi in aois:
    for year in years:
        if ('point_conception' != aoi) or (year != 2016):  #there's no data for Point Conception on 2016
            aoi_col.append(aoi)
            year_col.append(year)

            fp = os.path.join(os.getcwd(),
                              'polygons_from_naip_images',
                              aoi+'_polygons',
                              aoi+'_polygons_'+str(year),
                              aoi+'_polygons_'+str(year)+'.shp')

            df = gpd.read_file(fp)

            n_polygons.append(len(df))
            n_iceplant.append(len(df.loc[df['iceplant']==1]))
            
            
            itemid = df.naip_id[0]
            crs = sr.get_crs_from_itemid(itemid)
            df.to_crs(crs, inplace=True)
            df['area'] = [poly.area for poly in df.geometry] # in m^2

            area_ice.append(sum(df.loc[df['iceplant']==1].area))
            total_area.append(sum(df.area))

In [71]:
D = { 
    'aoi': aoi_col, 
    'year': year_col,
    'n_polys' : n_polygons,
    'total_area_m2' : total_area,    
    'n_ice_polys' : n_iceplant,
    'ice_area_m2' : area_ice
    }
df_all_polys = pd.DataFrame(D)
df_all_polys['n_non_ice_polys'] = df_all_polys.n_polys - df_all_polys.n_ice_polys
df_all_polys['non_ice_area_m2'] = (df_all_polys.total_area_m2 - df_all_polys.ice_area_m2)
df_all_polys

,aoi,year,n_polys,total_area_m2,n_ice_polys,ice_area_m2,n_non_ice_polys,non_ice_area_m2
0,campus_lagoon,2012,66,32337.253957,32,10378.999518,34,21958.254439
1,campus_lagoon,2014,50,30535.397580,24,7280.099361,26,23255.298219
2,campus_lagoon,2016,56,27186.644947,33,5788.021732,23,21398.623215
3,campus_lagoon,2018,91,191375.361864,30,8323.143326,61,183052.218537
4,campus_lagoon,2020,55,160123.254631,25,6977.543117,30,153145.711514
5,carpinteria,2012,78,39359.278111,42,5571.354104,36,33787.924007
6,carpinteria,2014,64,35013.242801,31,3384.164299,33,31629.078502
7,carpinteria,2016,69,33688.743153,34,2311.551738,35,31377.191415
8,carpinteria,2018,101,37789.405042,64,4305.077052,37,33484.327990
9,carpinteria,2020,106,41595.071842,68,5385.474477,38,36209.597364


In [32]:
sum(df_all_polys.n_ice_polys)

597

In [33]:
sum(df_all_polys.n_polys)

1203

In [77]:
sum(df_all_polys.ice_area_m2)/10**6

0.15407772017280175

In [76]:
sum(df_all_polys.non_ice_area_m2)/10**6

1.1350356337074012